In [2]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from glob import glob
import shutil

In [3]:
#data_dir = "/media/minjee/4970a4b3-9bec-42aa-8022-ddff6e7b8f80/"
#img_dir = os.path.join(data_dir, "labeled/")
data_dir = os.getcwd()
img_dir = os.path.join(data_dir, 'images/')
excel_path = os.path.join(os.getcwd(), 'hh_label_all.xlsx')

df_ex = pd.read_excel(excel_path)

In [3]:
"""
Seperating raw images into 3 categories; surgery, anesthesia, still(useless images) 
"""
# classes directories
# /images
### /surgery
### /anesthesia
###    /handhygiene (target)
### /still
def get_classes(cls, df):
    classes = []
    for col in df_ex.columns:
        if cls in col:
            classes.append(col)
            
    subclasses = {}
    if not classes:
        return

    for cls in classes: # subclass
        subclasses[cls] = df[cls].iloc[0] 
    return subclasses

            
def sort_frames_into_categories(img_dir):
    cnt = 0
    images = glob(img_dir+'*.jpg')
    images.sort()
    
    for image in tqdm(images): 
        img_name= image.split('/')[-1].split('.')[0]
        
        video_id = int(img_name.split('_')[0])
        frame_num = img_name[-6:] # ex: 000001
        df = df_ex[df_ex['video_id'] == video_id]

        for phase in ['surgery', 'anesthesia', 'still']:
            frames = get_classes(phase, df)
            
            for procedure in sorted(frames.keys(), reverse=True):
                if frames[procedure] is np.nan:
                    continue
                
                lists = frames[procedure].strip().split(',')
                for el in lists:
                    start = el.strip().split('-')[0]
                    end = el.strip().split('-')[1]

                    if int(frame_num) in range(int(start), int(end)+1):
                        if not os.path.exists(image):
                            continue

                        #print("image: {}, procedure: {}".format(os.path.basename(image), procedure))

                        dst_path = os.path.join(data_dir, 'images/'+procedure)
                        if not os.path.exists(dst_path):
                            os.mkdir(dst_path)
                        shutil.move(image, dst_path)
                        #cnt += 1
                        break
                        
    return

In [4]:
sort_frames_into_categories(img_dir)

0it [00:00, ?it/s]


In [14]:
input_dir = os.path.join(data_dir, 'images/anesthesia/handhygiene')
video_dir = os.path.join(data_dir, "videos/anesthesia/handhygiene")

def get_last_video_id():
    videos = glob(os.path.join(video_dir, "*.avi"))
    v_last = sorted(videos, key=os.path.getmtime)[-1]
    return os.path.splitext(os.path.basename(v_last))[0].split('_')[-1]
        

def images_to_video():
    cnt = get_last_video_id()
    print(cnt)
    return
    for row in df_ex.values:
        vname = row[1] #video_name
        target = row[6] #anesthesia/handhygiene
        fps = row[8] #fps
        
        if target is np.nan:
            continue
            
        lists = target.strip().split(',')
        for el in lists:
            start = el.strip().split('-')[0]
            
            images_path = os.path.join(input_dir, vname+'_frames%06d.jpg')
            target_dir = os.path.join(video_dir, vname+'_'+str(cnt)+'.avi')
            if not os.path.exists(target_dir):
                cnt += 1
                !ffmpeg -r $fps -start_number $start -i $images_path -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p $target_dir -y
            

In [15]:
images_to_video()

18
